In [ ]:
# version 1
class KNN1(object):
    def __init__(self, k):
        self.k = k

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.tr_time = self.timetosec(X_train)
        self.timeorder = [i[0] for i in sorted(enumerate(self.tr_time), key=lambda x:x[1])]
        self.y_train = y_train        
        
    def timetosec(self, data): # convert time to seconds
        convert_sec = lambda t:(datetime.strptime(t,'%Y-%m-%d %H:%M:%S.%f')-datetime(1970,1,1)).total_seconds()
        time = map(convert_sec ,data[:,2].tolist())
        return time
    
    def partition(self, vector, left, right, pivotIndex):
        pivotValue = vector[pivotIndex]
        vector[pivotIndex], vector[right] = vector[right], vector[pivotIndex]  # Move pivot to end
        storeIndex = left
        for i in range(left, right):
            if vector[i] < pivotValue:
                vector[storeIndex], vector[i] = vector[i], vector[storeIndex]
                storeIndex += 1
        vector[right], vector[storeIndex] = vector[storeIndex], vector[right]  # Move pivot to its final place
        return storeIndex

    def _select(self, vector, left, right, k):
        "Returns the k-th smallest, (k >= 0), element of vector within vector[left:right+1] inclusive."
        while True:
            pivotIndex = random.randint(left, right)     # select pivotIndex between left and right
            pivotNewIndex = self.partition(vector, left, right, pivotIndex)
            pivotDist = pivotNewIndex - left
            if pivotDist == k:
                return vector[pivotNewIndex]
            elif k < pivotDist:
                right = pivotNewIndex - 1
            else:
                k -= pivotDist + 1
                left = pivotNewIndex + 1

    def select(self, vector, k, left=None, right=None):
        """
        Returns the k-th smallest, (k >= 0), element of vector within vector[left:right+1].
        left, right default to (0, len(vector) - 1) if omitted.
        """
        if vector:
            if left is None:
                left = 0
            lv1 = len(vector) - 1
            if right is None:
                right = lv1
            if k >= lv1+1:
                return max(vector)
            else:
                return self._select(vector, left, right, k)
        return float('nan')
    
    
    def predict(self, X_test):
        '''
        This function does ....
        
        X_test
        
        Return:
        
        corner cases 
        '''
        if not len(X_test):
            print "X_test cannot be empty"
            return []
        
        y_test = np.zeros(len(X_test))
        test_time = self.timetosec(X_test)
        for i,t in enumerate(test_time): # test points
            distances = []
            for j in self.timeorder: # training points
                # compare datetime
                if self.tr_time[j] >= t: 
                    break
                # calculate euclidean distance
                dis = np.sum(np.square(np.subtract(X_test[i][:2], self.X_train[j][:2])))
                # add it to list of distances
                distances.append([dis, j])
            k_dis = self.select([m[0] for m in distances], self.k-1)
            idx = [d[1] for d in distances if d[0] <= k_dis]
            # get mean of k nearest neighbors
            y_test[i] = np.mean(self.y_train[[idx]])
        return y_test
    
    def mrae(self, X_test, y_test): 
        # performance measured in Median Relative Absolute Error
        pred = self.predict(X_test)
        notnan = ~np.isnan(pred)
        return np.median(abs(pred[notnan] - y_test[notnan])/y_test[notnan])

In [ ]:
# version 2
class KNN2(object):
    def __init__(self, k):
        self.k = k

    def fit(self, X_train, y_train):
        self.order = X_train[:,2].argsort()
        self.tr_time = self.timetosec(X_train)
        self.X_train = X_train
        self.y_train = y_train
#         self.X_train = X_train
#         self.timeorder = [i[0] for i in sorted(enumerate(self.tr_time), key=lambda x:x[1])]
#         self.y_train = y_train        
        
    def timetosec(self,data): # convert time to seconds
        convert_sec = lambda t:(datetime.strptime(t,'%Y-%m-%d %H:%M:%S.%f')-datetime(1970,1,1)).total_seconds()
        time = map(convert_sec ,data[:,2].tolist())
        return time
    
    def predict(self,X_test):
        y_test = np.zeros(len(X_test))
        test_time = self.timetosec(X_test)
        for i,t in enumerate(test_time): # test points
            distances = []
            for j in self.order: # training points
                # compare datetime
                if self.tr_time[j] >= t: 
                    break
                # calculate euclidean distance
                dis = np.sum(np.square(np.subtract(X_test[i][:2], self.X_train[j][:2])))
                # add it to list of distances
                distances.append([dis, j])
            # sort the list
            distances = sorted(distances)
            # make a list of the k neighbors' targets
            index = [ele[1] for ele in distances[:self.k]]
            # get mean of k nearest neighbors
            y_test[i] = np.mean(self.y_train[[index]])
        return y_test
    
    def mrae(self, X_test, y_test): 
        # performance measured in Median Relative Absolute Error
        pred = self.predict(X_test)
        notnan = ~np.isnan(pred)
        return np.median(abs(pred[notnan] - y_test[notnan])/y_test[notnan])

In [ ]:
# version 0

class KNN(object):
    def __init__(self, k):
        self.k = k

    def fit(self, X_train, y_train):
        order = X_train[:,2].argsort()
        self.X_train = X_train[order]
        self.y_train = y_train[order]
    
    def predict(self,X_test):
        y_test = np.zeros(len(X_test))
        for i in range(len(X_test)): # test points
            distances = []
            for j in range(len(self.X_train)): # training points
                # compare datetime
                if datetime.strptime(X_test[i][2],'%Y-%m-%d %H:%M:%S.%f') <= \
                datetime.strptime(self.X_train[j][2],'%Y-%m-%d %H:%M:%S.%f'): 
                    break
                # calculate euclidean distance
                distance = np.sum(np.square(np.subtract(X_test[i][:2], self.X_train[j][:2])))
                # add it to list of distances
                distances.append([distance, j])
            # sort the list
            distances = sorted(distances)
            # make a list of the k neighbors' targets
            index = [ele[1] for ele in distances[:self.k]]
            # get mean of k nearest neighbors
            y_test[i] = np.mean(self.y_train[[index]])
        return y_test
 
    def mrae(self, X_test, y_test):
        pred = self.predict(X_test)
        notnan = ~np.isnan(pred)
        return np.median(abs(pred[notnan] - y_test[notnan])/y_test[notnan])